In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import label_binarize
import pandas as pd

In [ ]:
# Set the paths
dataset_dir = '/content/drive/MyDrive/train'  # Update this with your actual dataset path

# ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Use 20% of the data for validation
)

# Load images from directory and apply transformations
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Models expect 224x224 images
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),  # Models expect 224x224 images
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Check the number of classes
num_classes = len(train_generator.class_indices)

print(f'Number of classes: {num_classes}')
print('Class indices:', train_generator.class_indices)

Found 5652 images belonging to 7 classes.
Found 1409 images belonging to 7 classes.
Number of classes: 7
Class indices: {'common_daisy': 0, 'coreopsis': 1, 'dandelion': 2, 'rose': 3, 'sunflower': 4, 'tulip': 5, 'water_lily': 6}


In [ ]:
# Load the MobileNetV2 model with pretrained weights, excluding the top layers
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
mobilenet_model = Sequential([
    mobilenet_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Use the correct number of classes
])

# Freeze the layers of MobileNetV2
for layer in mobilenet_base.layers:
    layer.trainable = False

# Compile the model
mobilenet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
mobilenet_history = mobilenet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Evaluate the model
mobilenet_loss, mobilenet_accuracy = mobilenet_model.evaluate(val_generator)
print(f'MobileNetV2 Validation Accuracy: {mobilenet_accuracy*100:.2f}%')

9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/20
176/176 [==============================] - 3635s 21s/step - loss: 3.0654 - accuracy: 0.7795 - val_loss: 0.3097 - val_accuracy: 0.9077
Epoch 2/20
176/176 [==============================] - 136s 773ms/step - loss: 0.5063 - accuracy: 0.8512 - val_loss: 0.2564 - val_accuracy: 0.9190
Epoch 3/20
176/176 [==============================] - 136s 774ms/step - loss: 0.4589 - accuracy: 0.8637 - val_loss: 0.2443 - val_accuracy: 0.9347
Epoch 4/20
176/176 [==============================] - 118s 673ms/step - loss: 0.4117 - accuracy: 0.8744 - val_loss: 0.2446 - val_accuracy: 0.9318
Epoch 5/20
176/176 [==============================] - 117s 664ms/step - loss: 0.4323 - accuracy: 0.8810 - val_loss: 0.2594 - val_accuracy: 0.9197
Epoch 6/20
176/176 [==============================] - 119s 677ms/step - loss: 0.3903 - accuracy: 0.8865 - val_loss: 0.2353 - val_accuracy: 0.9325
Epoch 7/20
176/176 [==============================] - 116s 661

In [ ]:
# Load the InceptionV3 model with pretrained weights, excluding the top layers
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
inception_model = Sequential([
    inception_base,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Use the correct number of classes
])

# Freeze the layers of InceptionV3
for layer in inception_base.layers:
    layer.trainable = False

# Compile the model
inception_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
inception_history = inception_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Evaluate the model
inception_loss, inception_accuracy = inception_model.evaluate(val_generator)
print(f'InceptionV3 Validation Accuracy: {inception_accuracy*100:.2f}%')

In [ ]:
# Load the VGG16 model with pretrained weights, excluding the top layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model
vgg_model = Sequential([
    vgg_base,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Use the correct number of classes
])

# Freeze the layers of VGG16
for layer in vgg_base.layers:
    layer.trainable = False

# Compile the model
vgg_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
vgg_history = vgg_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Evaluate the model
vgg_loss, vgg_accuracy = vgg_model.evaluate(val_generator)
print(f'VGG16 Validation Accuracy: {vgg_accuracy*100:.2f}%')

In [ ]:
# Plot training and validation history
def plot_history(histories, model_names):
    for history, model_name in zip(histories, model_names):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title(f'Training and validation accuracy for {model_name}')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs, loss, 'r', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title(f'Training and validation loss for {model_name}')
        plt.legend()

        plt.show()

histories = [vgg_history, inception_history, mobilenet_history]
model_names = ['VGG16', 'InceptionV3', 'MobileNetV2']

plot_history(histories, model_names)

# Function to plot confusion matrix and classification report
def plot_confusion_matrix_and_classification_report(model, val_generator, model_name):
    # Predict the classes
    Y_pred = model.predict(val_generator)
    y_pred = np.argmax(Y_pred, axis=1)

    # True classes
    y_true = val_generator.classes

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index = val_generator.class_indices.keys(), columns = val_generator.class_indices.keys())

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix for {model_name}')
    plt.ylabel('Actual Class')
    plt.xlabel('Predicted Class')
    plt.show()

    # Classification report
    cr = classification_report(y_true, y_pred, target_names=val_generator.class_indices.keys())
    print(f'Classification Report for {model_name}:\n', cr)

# Plot for each model
plot_confusion_matrix_and_classification_report(vgg_model, val_generator, 'VGG16')
plot_confusion_matrix_and_classification_report(inception_model, val_generator, 'InceptionV3')
plot_confusion_matrix_and_classification_report(mobilenet_model, val_generator, 'MobileNetV2')

# Function to plot precision-recall curve
def plot_precision_recall_curve(model, val_generator, model_name):
    # Predict the probabilities
    Y_pred = model.predict(val_generator)

    # True classes (one-hot encoded)
    y_true = label_binarize(val_generator.classes, classes=list(range(num_classes)))

    plt.figure(figsize=(10, 7))

    for i in range(num_classes):
        precision, recall, _ = precision_recall_curve(y_true[:, i], Y_pred[:, i])
        plt.plot(recall, precision, lw=2, label=f'Class {i} ({list(val_generator.class_indices.keys())[i]})')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall curve for {model_name}')
    plt.legend(loc='best')
    plt.show()

# Plot for each model
plot_precision_recall_curve(vgg_model, val_generator, 'VGG16')
plot_precision_recall_curve(inception_model, val_generator, 'InceptionV3')
plot_precision_recall_curve(mobilenet_model, val_generator, 'MobileNetV2')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_history(histories, model_names):
    for history, model_name in zip(histories, model_names):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(len(acc))

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(epochs, acc, 'r', label='Training accuracy')
        plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
        plt.title(f'Training and validation accuracy for {model_name}')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs, loss, 'r', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title(f'Training and validation loss for {model_name}')
        plt.legend()

        plt.show()

histories = [vgg_history, mobilenet__history, inception_history]
model_names = ['VGG16', 'MobileNetV2', 'InceptionV3']

plot_history(histories, model_names)